# Символьное дифференцирование

## Порядок сдачи домашнего

Под каждое домашнее вы создаете отдельную ветку куда вносите все изменения в рамках домашнего. Как только домашнее готово - создаете пулл реквест (обратите внимание что в пулл реквесте должны быть отражены все изменения в рамках домашнего). Ревьювера назначаете из таблицы - https://docs.google.com/spreadsheets/d/1vK6IgEqaqXniUJAQOOspiL_tx3EYTSXW1cUrMHAZFr8/edit?gid=0#gid=0
Перед сдачей проверьте код, напишите тесты. Не забудьте про PEP8, например, с помощью flake8. Задание нужно делать в jupyter notebook.

**Дедлайн - 18 ноября 10:00**

Символьное дифференцирование это инструмент для автоматического вывода формул производных, который открывает возможности для анализа сложных функций, оптимизации процессов и работы с уравнениями. Мы уже на многих занятиях сталкивались с этой темой - давайте попробуем реализовать собственное!

## Выражение

Создадим основной класс `Expr`, от которого будут наследоваться различные типы выражений, такие как константы, переменные, суммы, произведения и другие. Класс должен содержать методы:
* `__call__`, который будет вычислять значение выражения, используя переданный ему контекст (словарь, связывающий имена переменных с их значениями).
* `d`, принимающий имя переменной, по которой требуется вычислить производную, и возвращающий выражение, представляющее производную по этой переменной.

Эти методы нужно будет переопределить в каждом из подклассов для корректного выполнения операций.

In [ ]:
class Expr:
    def __call__(self, **context):
        pass
    
    def __neg__(self):
        return Product(Const(-1), self)

    def __pos__(self):
        return self
    
    def __add__(self, other):
        return Sum(self, other)

    def __sub__(self, other):
        return Sum(self, other.__neg__())

    def __mul__(self, other):
        return Product(self, other)

    def __truediv__(self, other):
        return Fraction(self, other)
    
    def d(self, var):
        pass

Создайте классы для двух видов выражений: `Const`, представляющий константу, и` Var`, представляющий переменную. Чтобы упростить использование, вместо обращения к конструкторам этих классов, будем использовать их однобуквенные сокращённые обозначения.

**Пример использования:**
```python
V = Var
C = Const

C(5)()
5
C(5).d(V("x"))()
0
V("x")(x=5)
5
V("x").d(V("y"))(x=5)
0
V("x").d(V("x"))(x=5)
1
```

In [ ]:
class Const(Expr):
    def __init__(self, const):
        self.const = const

    def __call__(self, **context):
        return self.const

    def d(self, var):
        return Const(0)

class Var(Expr):
    def __init__(self, name):
        self.name = name

    def __call__(self, **context):
        return context[self.name]

    def d(self, var):
        if self.name == var.name:
            return Const(1)
        return Const(0)

In [ ]:
# example
V = Var
C = Const

print(C(5)(),
C(5).d(V("x"))(),
V("x")(x=5),
V("x").d(V("y"))(x=5),
V("x").d(V("x"))(x=5))

In [ ]:
# test

def all_test_1():
    # test 1
    expr = Const(3) + Const(4)
    assert expr() == 7, f"Expected 7, but got {expr()}"

    expr = Const(3) * Const(4)
    assert expr() == 12, f"Expected 12, but got {expr()}"

    # test 2
    expr = Var("x") + Const(4)
    assert expr(x=5) == 9, f"Expected 9, but got {expr(x=5)}"
    assert expr.d(Var("x"))(x=5) == 1, f"Expected 1, but got {expr.d(Var('x'))()(x=5)}"

    # test 3
    expr = Var("x") * Var("x") + Const(3) * Var("x") + Const(2)
    assert expr(x=2) == 12, f"Expected 12, but got {expr(x=2)}"
    expr_d = expr.d(Var("x"))
    assert expr_d(x=2) == (2 * 2 + 3), f"Expected 7, but got {expr_d(x=2)}"

    # test 4
    expr = Var("x") * Var("y")
    assert expr(x=3, y=4) == 12, f"Expected 12, but got {expr(x=3, y=4)}"
    assert expr.d(Var("x"))(x=3, y=4) == 4, f"Expected 4, but got {expr.d(Var('x'))(x=3, y=4)}"
    assert expr.d(Var("y"))(x=3, y=4) == 3, f"Expected 3, but got {expr.d(Var('y'))(x=3, y=4)}"

    return "everything is working fine!"

print(all_test_1())

## Бинарные операции

Создайте классы для бинарных операций: `Sum`, `Product` и `Fraction`. Поскольку бинарные операции определяются двумя операндами, их конструктор будет одинаковым для всех этих классов. Поэтому его можно вынести в отдельный базовый класс, чтобы избежать дублирования кода.

In [ ]:
class BinOp(Expr):
    def __init__(self, expr1, expr2):
        self.expr1, self.expr2 = expr1, expr2

Реализуйте `Sum` для суммирования, `Product` для умножения и `Fraction` для деления.

**Пример использования:**

```python
Sum(V("x"), Fraction(V("x"), V("y")))(x=5, y=2.5)
7.0
Fraction(Sum(C(5), V("y")), Product(V("x"), V("y")))(x=1, y=2)
3.5
Fraction(Sum(C(5), V("y")), Product(V("x"), V("y"))).d(V("x"))(x=1, y=2)
-3.5
Fraction(Sum(C(5), V("y")), Product(V("x"), V("y"))).d(V("y"))(x=1, y=2)
-1.25
```

In [ ]:
class Sum(BinOp):
    def __call__(self, **context):
        return self.expr1(**context) + self.expr2(**context)

    def d(self, var):
        return Sum(self.expr1.d(var), self.expr2.d(var))

class Product(BinOp):
    def __call__(self, **context):
        return self.expr1(**context) * self.expr2(**context)

    def d(self, var):
        return Sum(
            Product(self.expr1.d(var), self.expr2),
            Product(self.expr1, self.expr2.d(var))
            )


class Fraction(BinOp):
    def __call__(self, **context):
        return self.expr1(**context) / self.expr2(**context)

    def d(self, var):
        up_expression = Sum(
                        Product(self.expr1.d(var), self.expr2),
                        Product(Const(-1), Product(self.expr1, self.expr2.d(var)))
                        )
        down_expression = Product(self.expr2, self.expr2)
        return Fraction(up_expression, down_expression)


In [ ]:
# example

print(Sum(V("x"), Fraction(V("x"), V("y")))(x=5, y=2.5),
Fraction(Sum(C(5), V("y")), Product(V("x"), V("y")))(x=1, y=2),
Fraction(Sum(C(5), V("y")), Product(V("x"), V("y"))).d(V("x"))(x=1, y=2),
Fraction(Sum(C(5), V("y")), Product(V("x"), V("y"))).d(V("y"))(x=1, y=2))

In [ ]:
# test

def all_test_2():
    # test 1
    expr = Sum(Const(3), Const(4))
    assert expr() == 7, f"Expected 7, but got {expr()}"
    expr = Sum(Var("x"), Const(4))
    assert expr(x=5) == 9, f"Expected 9, but got {expr(x=5)}"

    # test 2
    expr = Product(Const(3), Const(4))
    assert expr() == 12, f"Expected 12, but got {expr()}"
    expr = Product(Var("x"), Var("y"))
    assert expr(x=3, y=4) == 12, f"Expected 12, but got {expr(x=3, y=4)}"

    # test 3
    expr = Fraction(Var("x"), Const(2))
    assert expr(x=6) == 3, f"Expected 3, but got {expr(x=6)}"
    expr = Fraction(Sum(Const(1), Var("x")), Var("y"))
    assert expr(x=1, y=2) == 1, f"Expected 1, but got {expr(x=1, y=2)}"

    # test 4
    expr = Sum(Var("x"), Const(4))
    expr_d = expr.d(Var("x"))
    assert expr_d(x=5) == 1, f"Expected 1, but got {expr_d(x=5)}"

    # test 5
    expr = Product(Var("x"), Var("y"))
    expr_dx = expr.d(Var("x"))
    expr_dy = expr.d(Var("y"))
    assert expr_dx(x=3, y=4) == 4, f"Expected 4, but got {expr_dx(x=3, y=4)}"
    assert expr_dy(x=3, y=4) == 3, f"Expected 3, but got {expr_dy(x=3, y=4)}"

    # test 6
    expr = Fraction(Sum(Const(6), Var("y")), Product(Var("x"), Var("y")))
    expr_dx = expr.d(Var("x"))
    expr_dy = expr.d(Var("y"))
    assert expr_dx(x=2, y=3) == -0.75, f"Expected -0.75, but got {expr_dx(x=2, y=3)}"
    assert expr_dy(x=3, y=2) == -0.5, f"Expected -0.5, but got {expr_dy(x=3, y=2)}"

    return "everything is working fine!"

print(all_test_2())

## Перегрузка операторов

Добавьте перегрузку операторов в базовых класс `Expr`. Обратите что в классах мы можем тоже заменить на использование операторов.
```python  
-e         e.__neg__()
+e         e.__pos__()
e1 + e2    e1.__add__(e2)
e1 - e2    e1.__sub__(e2)
e1 * e2    e1.__mul__(e2)
e1 / e2    e1.__truediv__(e2)
```

**Пример использования:**

```python
(V("x") * V("x") / V("y"))(x=5, y=2.5)
10.0
```

In [ ]:
# example

(V("x") * V("x") / V("y"))(x=5, y=2.5)

In [ ]:
# test

def all_test_3():
    # test 1
    expr = Const(3) + Const(4)
    assert expr() == 7, f"Expected 7, but got {expr()}"

    expr = Const(3) - Const(1)
    assert expr() == 2, f"Expected 2, but got {expr()}"

    expr = Const(3) * Const(4)
    assert expr() == 12, f"Expected 12, but got {expr()}"

    expr = Const(8) / Const(2)
    assert expr() == 4, f"Expected 4, but got {expr()}"

    # test 2
    expr = (Var("x") * Var("x")) / Var("y")
    assert expr(x=5, y=2.5) == 10.0, f"Expected 10.0, but got {expr(x=5, y=2.5)}"

    expr = Var("x") + Var("y") - Var("z")
    assert expr(x=5, y=3, z=1) == 7, f"Expected 7, but got {expr(x=5, y=3, z=1)}"

    # test 3
    expr = Var("x") * Var("x")
    expr_d = expr.d(Var("x"))
    assert expr_d(x=3) == 6, f"Expected 6, but got {expr_d(x=3)}"

    expr = Var("x") / Var("y")
    expr_dx = expr.d(Var("x"))
    expr_dy = expr.d(Var("y"))
    assert expr_dx(x=3, y=4) == 0.25, f"Expected 0.25, but got {expr_dx(x=3, y=4)}"
    assert expr_dy(x=3, y=4) == -0.1875, f"Expected -0.1875, but got {expr_dy(x=3, y=4)}"

    return "everything is working fine!"

print(all_test_3())

## Метод Ньютона-Рафсона

Напишите функцию `newton_raphson`, которая принимает дифференцируемую функцию  $f$  от переменной  $x$ , начальное приближение  $x_0$ , и положительное число  $\epsilon$ , задающее точность вычислений. Функция должна возвращать значение  $x$ , при котором  $f(x)$  становится равным нулю. Метод Ньютона-Рафсона выполняет итеративный поиск корня функции  $f(x)$ , начиная с начального значения  $x_0$ , и использует правило  
$$x_{n+1} = x_n - \frac{f(x_n)}{f{\prime}(x_n)}$$  
для обновления  $x$  на каждом шаге. Итерации продолжаются до тех пор, пока условие остановки  $|x_{n+1} - x_n| \leq \epsilon$  не будет выполнено.

**Пример использования:**

```python
x = Var("x")
f = Const(-5) * x * x * x * x * x + Const(3) * x + Const(2)
zero = newton_raphson(f, 0.5, eps=1e-4)
zero, f(x=zero)
(1.000000000001132, -2.490496697760136e-11)
```

In [ ]:
def newton_raphson(expr, x0, eps=1e-4):
    x_n = x0
    while True:
        f = expr(x=x_n)
        df = expr.d(Var("x"))(x=x_n)
        if df == 0: 
            raise Exception("newton's method is not suitable")
        x_n1 = x_n - f/df
        if abs(x_n1 - x_n) <= eps:
            break
        x_n = x_n1
    return x_n

In [ ]:
# example

x = Var("x")
f = Const(-5) * x * x * x * x * x + Const(3) * x + Const(2)
zero = newton_raphson(f, 0.5, eps=1e-4)
print(zero, f(x=zero))

In [ ]:
# test

def all_test_4():
    # test 1: x^2 - 2
    expr = Var("x") * Var("x") - Const(2)
    root = newton_raphson(expr, x0=1.0, eps=1e-4)
    assert expr(x=root) <= 1e-4, f"Expected approximately {2**0.5}, but got {root}"

    # test 2: x^3 - x
    expr = Var("x") * Var("x") * Var("x") - Var("x")
    root = newton_raphson(expr, x0=0.5, eps=1e-4)
    assert expr(x=root) <= 1e-4, f"Expected approximately 1, but got {root}"

    # test 3: 2x - 4
    expr = Const(2) * Var("x") - Const(4)
    root = newton_raphson(expr, x0=0, eps=1e-4)
    assert expr(x=root) <= 1e-4, f"Expected approximately 2, but got {root}"

    # test 4: x^2 + 1, no real root
    expr = Var("x") * Var("x") + Const(1)
    try:
        newton_raphson(expr, x0=0, eps=1e-4)
        assert False, "Expected an exception for no real roots"
    except Exception as e:
        assert str(e) == "newton's method is not suitable", f"Unexpected exception message: {str(e)}"

    return "everything is working fine!"

print(all_test_4())